# <b><u> CONCURRENT AUDIT AUTOMATION TOOL </u></b>

## <u> Importing necessary libraries, packages, toolbox and credentials for GCP </u>

In [80]:
import tkinter as tk
from tkinter import font
from tkinter import filedialog as fd

import sys 
import os 
import io
import re

import numpy as np
import pandas as pd

from PIL import Image, ImageOps
from pdf2image import convert_from_path
from difflib import SequenceMatcher

credential_path = 'E:\GCP\macbook-2c718e707810.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

from toolbox import *
#from toolbox import detect_document, find_ref, amt, currency, givedates, changeformat

## <u> Functions linked with the buttons of GUI </u>

In [81]:
# Functions for storing the path of PDF and System Dump
def pdf():
    global pdf_path
    pdf_path = fd.askopenfilename()
    pdfpath = tk.Label(frame1, text = pdf_path)
    pdfpath.place(relx = 0.26, rely = 0.3, relwidth = 0.74, relheight = 0.2)
    
    
def dump():
    global dmp_path
    dmp_path = fd.askopenfilename()
    syspath = tk.Label(frame1, text = dmp_path)
    syspath.place(relx = 0.26, rely = 0.53, relwidth = 0.74, relheight = 0.2)

In [82]:
# Function for processing the files and storing the results in the form of EXCEL File 
def process():
    # Get from global variables
    pdf_name = pdf_path
    system_dump = dmp_path
    
    img_name = 'img'
    
    print("<<BREAK - POINT>>")

    pages = convert_from_path(pdf_name, 500)
    border = 150
    image_counter = 1
    for page in pages: 
        filename = img_name+str(image_counter)+".jpg"
        page = ImageOps.expand(page, border=border, fill = 'white')
        page.save(filename, 'JPEG') 
        image_counter = image_counter + 1 
    filelimit = image_counter-1

    data =[]
    for i in range(1, filelimit + 1): 
        filename = img_name+str(i)+".jpg"
        text = detect_document(filename)
        data.append(text)

    strf = ''
    for i in range(len(data)):
        for j in range(len(data[i])):
            for k in range(len((data[i])[j])):
                strf += ((((data[i])[j])[k]).strip(" "))
                strf += ' '
    data1 = strf
    data1 = data1.lower()
    
    
    ref_no = find_ref(data1)
    invoice_amt = amt(data1)
    cur_list = currency(data1)
    organiz_name = organisation(data)
    
    
    # Reading Excel file
    sys_dump = pd.read_excel(system_dump, index_col=0)
    idx = sys_dump.index[sys_dump['REF_ID'] == ref_no].tolist()

    sys_counter_party = (sys_dump.loc[idx[0]]['COUNTERPARTY_FULLNAME']).lower()
    sys_trade_date = (sys_dump.loc[idx[0]]['TRADE_DATE'])
    sys_settle_date = (sys_dump.loc[idx[0]]['SETTLEMENT_DATE'])
    sys_cur1 = (sys_dump.loc[idx[0]]['CURRENCY1']).lower()
    sys_cur2 = (sys_dump.loc[idx[0]]['CURRENCY2']).lower()
    sys_amt = float(sys_dump.loc[idx[0]]['NOTIONAL1'])
    # Stamp date also to be included
    
    # Flag will be raised when there is exception
    amt_f = 0
    ivd_f = 0
    pd_f = 0
    cur_f = 0
    org_f = 0
    stm_f = 0
    
    valid = 0
    
    # Processing for date
    td = changeformat(sys_trade_date)
    sd = changeformat(sys_settle_date)
    listx = givedates(data1)
    
    # Verification for Amount
    if(invoice_amt >= sys_amt):
        amt_f = 0
    else:
        amt_f = 1



    # Verification for currency
    if((sys_cur1 in cur_list) and (sys_cur2 in cur_list)):
        cur_f = 0
    else:
        cur_f = 1
        
        
    # Verification for organisation
    if(sys_counter_party in sub):
        org_f = 0
    else:
        org_f = 1


    # Trade(sys_dmp) - Invoice(doc) > 0
    delta1 = date(td[0], td[1], td[2]) - date(listx[0][0], listx[0][1], listx[0][2])
    if(delta1.days > 0):
        ivd_f = 0
    else:
         ivd_f = 1



    # Payment date(doc) - Settlement date > 0
    delta2 = date(listx[1][0], listx[1][1], listx[1][2]) - date(sd[0], sd[1], sd[2])
    if(delta2.days > 0):
        pd_f = 0
    else:
        pd_f = 1
        
    
    stamp_message = ''
    #  Stamp(doc) - Trade <= 15
    if(len(listx) >= 3):
        delta3 = date(listx[-1][0], listx[-1][1], listx[-1][2]) - date(td[0], td[1], td[2])
        if(delta3.days <= 15):
            stm_f = 0
        else:
            stamp_message = ' Stamp date did not match.'
            stm_f = 1
    else:
        stamp_message = ' Stamp not found.'
        stm_f = 1



    # Document verified or not
    if(amt_f == 0 and ivd_f == 0 and pd_f == 0 and cur_f == 0 and org_f == 0 and stm_f == 0):
        valid = 0
    else:
        valid = 1



    if(valid==1):
        validation = 'Not Verified !!'

        message = ''
        if(amt_f == 1):
            message += ' Amount,'
        if(ivd_f == 1):
            message += ' Invoice date,' 
        if(pd_f == 1):
            message += ' Payment date,'
        if(cur_f == 1):
            message += ' Currency,'
        if(org_f == 1):
            message += ' Organisation,'
        message +=' is/are not verified in the document. /n'
        message += stamp_message
        
    else:
        validation = 'Verified !!'
        message = 'All the fields are verified'
    
    dlist = datestring(data1)
    invdate = dlist[0]
    setdate = dlist[1]
    
    df1 = pd.read_excel("output.xlsx", index_col=None)
    df2 = pd.DataFrame({"Reference no.":[ref_no], 
                        "Invoice Date":[invdate.upper()],
                        "Settlement Date":[setdate.upper()],
                        "Counterparty Name":[sys_counter_party.upper()], 
                        "Amount":[invoice_amt],
                        "Verification":[validation.upper()],
                        "Message":[message,upper()]})
    df1 = df1.append(df2)
    df1.to_excel("output.xlsx")
    

In [83]:
# This function extracts the data attributes from the saved output file and display them
def extract():

    d0f = pd.read_excel("output.xlsx", index_col=None)

    ref_no = int(d0f.iloc[-1]['Reference no.'])
    invoice_date = str(d0f.iloc[-1]['Invoice Date'])
    firm_name = str(d0f.iloc[-1]['Counterparty Name'])
    settle_date = str(d0f.iloc[-1]['Settlement Date'])
    invoice_amt = float(d0f.iloc[-1]['Amount'])
    
    #root = tk.Toplevel()
    
    
    frame2 = tk.Frame(root, bg = frame_bg, bd = 10)
    frame2.place(relx = 0.01, rely = 0.32, relwidth = 0.98, relheight = 0.4)
    

    button6 = tk.Button(frame2, text = "EXTRACT", font=("Times", 15), bg='#2B4693', fg="white", command = extract)
    button6.place(relx = 0.3, rely = 0.82, relwidth = 0.4, relheight = 0.2)

    label4a = tk.Label(frame2, text = "REFERENCE NO.")
    label4a.place(relx = 0, rely = 0.01, relwidth = 0.25, relheight = 0.15)
    label4b = tk.Label(frame2, text = ref_no)
    label4b.place(relx = 0.26, rely = 0.01, relwidth = 0.73, relheight = 0.15)

    label5a = tk.Label(frame2, text = "INVOICE DATE")
    label5a.place(relx = 0, rely = 0.17, relwidth = 0.25, relheight = 0.15)
    label5b = tk.Label(frame2, text = invoice_date)
    label5b.place(relx = 0.26, rely = 0.17, relwidth = 0.73, relheight = 0.15)

    label6a = tk.Label(frame2, text = "FIRM'S NAME")
    label6a.place(relx = 0, rely = 0.33, relwidth = 0.25, relheight = 0.15)
    label6b = tk.Label(frame2, text = firm_name)
    label6b.place(relx = 0.26, rely = 0.33, relwidth = 0.73, relheight = 0.15)

    label7a = tk.Label(frame2, text = "SETTLEMENT DATE")
    label7a.place(relx = 0, rely = 0.49, relwidth = 0.25, relheight = 0.15)
    label6b = tk.Label(frame2, text = settle_date)
    label6b.place(relx = 0.26, rely = 0.49, relwidth = 0.73, relheight = 0.15)

    label8a = tk.Label(frame2, text = "INVOICE AMOUNT")
    label8a.place(relx = 0, rely = 0.65, relwidth = 0.25, relheight = 0.15)
    label8b = tk.Label(frame2, text = invoice_amt)
    label8b.place(relx = 0.26, rely = 0.65, relwidth = 0.73, relheight = 0.15)

In [84]:
# This function displays the verification message and assessment report 
def verify_doc():
    
    d0f = pd.read_excel("output.xlsx", index_col=None)

    verifi = str(d0f.iloc[-1]['Verification'])
    message = str(d0f.iloc[-1]['Message'])
        
    #root = tk.Toplevel()
    
    verify = tk.Label(frame3, text = verifi)
    verify.place(relx = 0.26, rely = 0.01, relwidth = 0.73, relheight = 0.2)

    
    verifyasrpt = tk.Label(frame3, text = message)
    verifyasrpt.place(relx = 0.26, rely = 0.25, relwidth = 0.73, relheight = 0.4)
    

## <u> Graphical User Interface </u>

In [86]:
HEIGHT = 700
WIDTH = 650

root = tk.Toplevel()

frame_bg = '#232727'
but_lab = '#73788A'
canv = 'black'

root.title('Concurrent Audit Automation Software')

canvas = tk.Canvas(root, height = HEIGHT, width = WIDTH, bg=canv)
canvas.pack()



frame1 = tk.Frame(root, bg = frame_bg, bd = 10)
frame1.place(relx = 0.01, rely = 0.01, relwidth = 0.98, relheight = 0.3)


label1 = tk.Label(frame1, text = "UPLOAD DATA", font=("Times", 17), bg='#232727', fg="white" )
label1.place(relx = 0.3, rely = 0, relwidth = 0.4, relheight = 0.2)
#f = font.Font(label1, label1.cget("font"))
#f.configure(underline=True)
#label1.configure(font=f)

pdfbutton = tk.Button(frame1, text = "Document",  font=("Times", 15), bg=but_lab, fg="white", command = pdf)
pdfbutton.place(relx = 0, rely = 0.3, relwidth = 0.25, relheight = 0.2)

pdfpath = tk.Label(frame1)
pdfpath.place(relx = 0.26, rely = 0.3, relwidth = 0.74, relheight = 0.2)

sysbutton = tk.Button(frame1, text = "System Dump",  font=("Times", 15), bg=but_lab, fg="white", command = dump)
sysbutton.place(relx = 0, rely = 0.53, relwidth = 0.25, relheight = 0.2)

syspath = tk.Label(frame1)
syspath.place(relx = 0.26, rely = 0.53, relwidth = 0.74, relheight = 0.2)



process = tk.Button(frame1, text = "PROCESS", font=("Times", 15), bg='#2B4693', fg="white", command = process)
process.place(relx = 0.35, rely = 0.78, relwidth = 0.3, relheight = 0.25)



frame2 = tk.Frame(root, bg = frame_bg, bd = 10)
frame2.place(relx = 0.01, rely = 0.32, relwidth = 0.98, relheight = 0.4)

canvas_bg = tk.Canvas(frame2, bg="black", height=250, width=300)
canvas_bg.place(relx = 0.0, rely = 0, relwidth = 1, relheight = 0.9)


one = tk.PhotoImage(file = 'stock-1863880_960_720.png')
root.one = one  # to prevent the image garbage collected.
canvas_bg.create_image((0,0), image=one, anchor='nw') 


button6 = tk.Button(frame2, text = "EXTRACT", font=("Times", 15), bg='#2B4693', fg="white", command = extract)
button6.place(relx = 0.3, rely = 0.82, relwidth = 0.4, relheight = 0.2)



frame3 = tk.Frame(root, bg = frame_bg, bd = 10)
frame3.place(relx = 0.01, rely = 0.73, relwidth = 0.98, relheight = 0.3)

button7 = tk.Button(frame3, text = "Verify Document",  font=("Times", 15), bg=but_lab, fg="white", command = verify_doc)
button7.place(relx = 0, rely = 0.01, relwidth = 0.25, relheight = 0.2)

verify = tk.Label(frame3)
verify.place(relx = 0.26, rely = 0.01, relwidth = 0.73, relheight = 0.2)

aserpt = tk.Label(frame3, text = "Assessment Report",  font=("Times", 15), bg=but_lab, fg="white")
aserpt.place(relx = 0, rely = 0.25, relwidth = 0.25, relheight = 0.2)

verifyasrpt = tk.Label(frame3)
verifyasrpt.place(relx = 0.26, rely = 0.25, relwidth = 0.73, relheight = 0.4)

#asrp = tk.Button(frame3, text = "SAVE ASSESSMENT REPORT", font=("Times", 15), bg="#6D54EC", fg="black", command = save_excel)
#asrp.place(relx = 0.25, rely = 0.7, relwidth = 0.5, relheight = 0.15)




root.mainloop()